# importing libraries

In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import time
import cv2
import os

# function to detect mask

In [3]:
def detectMask(frame,maskNet):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cascade = 'haarcascade_frontalface_default.xml'
    faceCascade = cv2.CascadeClassifier(cascade)
    faces = faceCascade.detectMultiScale(gray,1.3,5)  
    coordinates = []
    predictions=[]
    for (x,y,w,h) in faces: 
        face = frame[y:y+h, x:x+h]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        coordinates.append((x,y,w,h))
        
        if len(face)>0:
            predictions = maskNet.predict(face)
    return coordinates,predictions

# loading the model

In [ ]:
model = "model_mask.h5"
# load the face mask detector model from disk
print("loading face mask detector model...")
maskNet = load_model(model)
print('MODEL LOADED')

# predicting the mask

In [26]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    
    coordinates,predictions = detectMask(frame,maskNet)
    for rect,pred in  zip(coordinates,predictions):
        (x,y,w,h) = rect
        (mask, withoutMask) = pred

        if mask > withoutMask:
            label = "Mask"  
            color = (0, 255, 0)
        else:
            label = "No Mask"
            color = (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1)==13:
        cap.release()
        break

cv2.destroyAllWindows()